In [227]:
import pandas as pd
import numpy as np

'''
Author: Enrico Ceccolini
    Drop the jobs with zero timepoints.
    Split between trainset and testset
'''

datadir = "/datasets/eurora_data/db1/"
datadir = "/datasets/eurora_data/db/" #hdd

#infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"



### select an interval from
## 2 settings Andrea
#infile_jobs = datadir + "april_long_jobs.csv"
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-04-30')
test_end_time = pd.to_datetime('2014-05-10')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow" # there are also short jobs

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-11-01')
#train_start_time = pd.to_datetime('2014-03-31')
#train_end_time = pd.to_datetime('2014-10-01')
#test_end_time = pd.to_datetime('2014-11-01')
#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_jobs_real_pow" # there are also short jobs

infile_jobs = datadir + "CPUs/" + "Alina" + "/Alina_jobs_real_pow" # there are also short jobs

#nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [228]:
jobs = pd.read_csv(infile_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(jobs.shape[0], interval_comment))

# clean user names
jobs['user'] = jobs.apply(lambda row: row['user'].split('@')[0], axis=1)


There are 336787 long jobs in the Andrea's interval


In [229]:
jobs.keys()


Index(['job_id_string', 'job_name', 'queue', 'start_time', 'run_start_time',
       'end_time', 'user', 'node_req', 'cpu_req', 'mem_req', 'time_req',
       'deleted', 'dependency', 'exit_status', 'pbs_exit_code', 'gpu_req',
       'mic_req', 'real_pow', 'ran_alone', 'real_pow_quality', 'n_2_1',
       'n_2_2', 'n_3_1', 'job_tot_timepoints', 'job_timepoints', 'good_nodes',
       'ncores_system', 'njobs_system', 'load_system'],
      dtype='object')

In [230]:
# no jobs with real_pow smaller than 0
print(jobs[jobs['real_pow'] <= 0]['real_pow'].shape[0])
print(jobs[jobs['real_pow'] == 0]['real_pow'].shape[0])


25292
25292


fill the nan time_req using wall times

In [231]:
jobs['time_req'].unique()

array([nan, '04:00', '01:00', '00:30', '08:00', '03:59', '02:00', '03:30',
       '00:10', '00:09', '03:00', '00:02', '00:04', '00:01', '00:05',
       '00:08', '00:15', '00:29', '00:35', '00:20', '07:59', '00:03',
       '06:00', '00:00', '01:30', '03:29', '02:30', '01:10', '01:20',
       '00:40', '--', '03:20', '01:29'], dtype=object)

In [232]:
# debug
jobs[jobs['time_req'].isnull()]['queue'].unique()

array(['parallel', 'debug', 'np_longpar', 'p_devel', 'visual', 'system',
       'R810265', 'R928065', 'R959715', 'R959716', 'R959717', 'R959718',
       'R959719', 'R959720', 'R959721', 'R959722', 'R1010998', 'R1011000',
       'R1013900', 'R1013903', 'R1013901', 'R1013904', 'R1013902',
       'R1010870', 'R1172721', 'R1172722', 'R1172723', 'R1172724',
       'R1172725', 'R1208877', 'R1201947', 'R1201948', 'R1201949',
       'R1201950', 'R1201951', 'R1201952', 'R1287258', 'R1287259',
       'R1287260', 'R1287269', 'R1287268', 'R1287270', 'R1315151',
       'R1297176', 'R1297177', 'R1297178', 'R1297179', 'R1297180',
       'R1297181', 'route', 'meteopar', 'R588271', 'R588272', 'R588273',
       'R588274', 'R588275', 'R588276', 'R588277', 'R588278', 'R588279',
       'R588280', 'R588281', 'R588282', 'R588283', 'R588642', 'R588286',
       'R588287', 'R588288', 'R588289', 'R588290', 'R588291', 'R588293',
       'R588294', 'R588295', 'R588296', 'R588297', 'R588298', 'R628233',
       'R629

In [233]:
jobs.shape[0]

336787

In [234]:
jobs.loc[jobs['time_req'] == '--', 'time_req'] = np.NaN
print(jobs[jobs['time_req'].isnull()].shape[0])

277130


In [235]:
#debug wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'debug'), 'time_req' ] = '00:30'
print(jobs[jobs['time_req'].isnull()].shape[0])

229501


In [236]:
#p_devel wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'p_devel'), 'time_req' ] = '01:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

229386


In [237]:
#parallel wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'parallel'), 'time_req' ] = '04:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

65106


In [238]:
#np_longpar wall time
jobs.loc[ (jobs['time_req'].isnull()) & (jobs['queue'] == 'np_longpar'), 'time_req' ] = '08:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

62842


In [239]:
#all the others wall time
jobs.loc[ jobs['time_req'].isnull(), 'time_req'] = '04:00'
print(jobs[jobs['time_req'].isnull()].shape[0])

0


In [240]:
jobs[(jobs['job_tot_timepoints'] > 10) & (jobs['real_pow_quality'] <= 0)]

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,ncores_system,njobs_system,load_system
job_id,,,,,,,,,,,,,,,,,,,,,
229803,1198859.node129,sl1407_dmc,parallel,2014-09-11 00:42:00,2014-09-11 21:36:44,2014-09-11 22:16:36,gbertain,2,32,29360128,...,0.0,2,0,0,956,956,0,453,136,0
229804,1198860.node129,sl1407_dmc,parallel,2014-09-11 00:42:00,2014-09-11 22:16:39,2014-09-11 22:55:43,gbertain,2,32,29360128,...,0.0,2,0,0,936,936,0,446,138,0
229982,1199090.node129,equil-29,parallel,2014-09-11 12:00:17,2014-09-11 16:07:19,2014-09-11 20:08:06,pcacciot,1,2,3145728,...,0.0,0,0,1,2889,2889,0,355,63,0
229983,1199091.node129,equil-30,parallel,2014-09-11 12:00:17,2014-09-11 20:08:08,2014-09-12 00:09:23,pcacciot,1,2,3145728,...,0.0,0,0,1,2895,2895,0,469,128,0
229996,1199110.node129,tpa18fb_20.csh,parallel,2014-09-11 12:40:38,2014-09-12 07:24:03,2014-09-12 10:22:05,cmorgill,1,16,1048576,...,0.0,0,0,1,2136,2136,0,146,10,0
230041,1199176.node129,J13,parallel,2014-09-11 14:59:52,2014-09-11 16:52:04,2014-09-11 20:51:25,fnegreir,1,4,4194304,...,0.0,0,0,1,2872,2872,0,380,74,0
230042,1199177.node129,J14,parallel,2014-09-11 14:59:52,2014-09-11 16:52:05,2014-09-11 20:51:26,fnegreir,1,4,4194304,...,0.0,0,0,1,2872,2872,0,380,74,0
230043,1199178.node129,J15,parallel,2014-09-11 14:59:52,2014-09-11 16:52:05,2014-09-11 20:51:25,fnegreir,1,4,4194304,...,0.0,0,0,1,2872,2872,0,380,74,0
230044,1199179.node129,J16,parallel,2014-09-11 14:59:53,2014-09-11 16:55:43,2014-09-11 20:55:00,fnegreir,1,4,4194304,...,0.0,0,0,1,2871,2871,0,394,80,0


In [241]:
print(jobs.shape[0])
jobs_q1 = jobs[jobs['real_pow_quality'] > 0]
jobs_q1 = jobs_q1[jobs_q1['real_pow'] > 0] 
print(jobs_q1.shape[0])

336787
311495


In [242]:
print(jobs[(jobs['real_pow_quality'] == 0) | (jobs['real_pow'] <= 0)].shape)
dropped_jobs = jobs[(jobs['real_pow_quality'] <= 0) | (jobs['real_pow'] <= 0)]

(25292, 29)


In [243]:
jobs.groupby(['user', 'job_name']).size().reset_index(name='counts')

,user,job_name,counts
0,a07smr01,A2REPO,109
1,a07smr01,ARCHIVE,2
2,a07smr01,ARKI,125
3,a07smr01,INIT,137
4,a07smr01,LM28_OPE,133
5,a07smr01,LM7_OPE,123
6,a07smr01,LM7_TST,108
7,a07smr01,LM_PRE,162
8,a07smr01,POSTPROCESS,122
9,a07smr01,SWAN_CAM,107


In [244]:
19978 / 25292

0.7898940376403606

In [245]:
1 - (311495 / 336787)

0.07509791054880388

In [246]:
# with jobs['real_pow_quality'] > 0
#165126

# with jobs['real_pow_quality'] >= 0
#311172

In [247]:
print("{} distinct users".format(jobs_q1['user'].unique().shape[0]))

274 distinct users


In [248]:
jobs_q1.groupby(['user', 'job_name']).size().shape[0]

15007

In [249]:
jobs_train = jobs_q1[pd.to_datetime(jobs_q1['end_time']) <= train_end_time]
jobs_train = jobs_train[pd.to_datetime(jobs_train['run_start_time']) >= train_start_time]

print("train set contains {} records".format(jobs_train.shape[0]))

jobs_test = jobs_q1[pd.to_datetime(jobs_q1['end_time']) <= test_end_time]
jobs_test = jobs_test[pd.to_datetime(jobs_test['run_start_time']) >= train_end_time]
print("test set contains {} records".format(jobs_test.shape[0]))

train set contains 74240 records
test set contains 7557 records


In [250]:
jobs_train.groupby(['user', 'job_name']).size().reset_index(name='counts')

,user,job_name,counts
0,a07smr01,A2REPO,41
1,a07smr01,ARKI,49
2,a07smr01,INIT,40
3,a07smr01,LM28_OPE,49
4,a07smr01,LM7_OPE,50
5,a07smr01,LM7_TST,41
6,a07smr01,LM_PRE,77
7,a07smr01,POSTPROCESS,48
8,a07smr01,SWAN_CAM,39
9,a07smr01,SWAN_EMR,39


In [251]:
jobs_test.groupby(['user', 'job_name']).size().reset_index(name='counts')

,user,job_name,counts
0,a07smr01,A2REPO,10
1,a07smr01,ARCHIVE,2
2,a07smr01,ARKI,13
3,a07smr01,INIT,11
4,a07smr01,LM28_OPE,22
5,a07smr01,LM7_OPE,11
6,a07smr01,LM7_TST,11
7,a07smr01,LM_PRE,10
8,a07smr01,POSTPROCESS,11
9,a07smr01,SWAN_CAM,11


In [252]:
jobs_test

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,real_pow_quality,n_2_1,n_2_2,n_3_1,job_tot_timepoints,job_timepoints,good_nodes,ncores_system,njobs_system,load_system
job_id,,,,,,,,,,,,,,,,,,,,,
85459,621877.node129,4a_corr_ramp,parallel,2014-04-29 08:27:06,2014-04-30 04:21:32,2014-04-30 04:21:35,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,364,63,0
85461,621879.node129,Run_4e,parallel,2014-04-29 08:27:18,2014-04-30 04:21:44,2014-04-30 04:21:46,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,364,63,0
85466,621884.node129,SMD_4e_08,parallel,2014-04-29 08:28:02,2014-04-30 04:22:22,2014-04-30 04:22:26,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,364,63,0
85470,621888.node129,Run-2_4a,parallel,2014-04-29 08:28:33,2014-04-30 04:22:42,2014-04-30 04:22:45,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,364,63,0
85472,621890.node129,Run_4a,parallel,2014-04-29 08:28:45,2014-04-30 04:22:53,2014-04-30 04:22:55,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,364,63,0
85483,621901.node129,4e_corr_ramp,parallel,2014-04-29 08:29:59,2014-04-30 04:23:51,2014-04-30 04:23:53,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,363,62,0
85487,621905.node129,Trial_Run_4a,parallel,2014-04-29 08:30:29,2014-04-30 04:24:16,2014-04-30 04:24:19,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,363,62,0
85619,622045.node129,Run-2_apo_ncode,parallel,2014-04-29 09:21:21,2014-04-30 04:25:07,2014-04-30 04:25:09,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,363,62,0
86061,622970.node129,Run_apo_ncode,parallel,2014-04-29 14:27:25,2014-04-30 04:25:45,2014-04-30 04:25:47,szia0000,1,2,8388608,...,1.000000,0,0,1,1,0,1,363,62,0


In [253]:
286478 + 25016

311494

In [254]:
print("{} unique jobs in the trainset".format(jobs_train.groupby(['user', 'job_name']).size().shape[0]))
print("{} unique jobs in the testset".format(jobs_test.groupby(['user', 'job_name']).size().shape[0]))

2943 unique jobs in the trainset
1190 unique jobs in the testset


In [255]:
job_new = jobs_test[~jobs_test['job_name'].isin(jobs_train['job_name'])]
print(job_new['job_name'].unique().shape)

(427,)


In [256]:
#jobs_test = jobs_test_cpy

check if there are some single core jobs that have overscale consume

In [257]:
jobs_to_remove = jobs_test[(jobs_test.real_pow > 300) & (jobs_test.cpu_req == 1)]['job_id_string']

In [258]:
jobs_to_remove.shape

(4,)

In [259]:
jobs_test_cpy = jobs_test

In [260]:
jobs_test = jobs_test[~jobs_test.job_id_string.isin(jobs_to_remove)]

In [261]:
jobs_test.shape

(7553, 29)

Random train and test set

In [262]:
data_set.shape

(10768, 29)

In [263]:
data_set.columns

Index(['job_id_string', 'job_name', 'queue', 'start_time', 'run_start_time',
       'end_time', 'user', 'node_req', 'cpu_req', 'mem_req', 'time_req',
       'deleted', 'dependency', 'exit_status', 'pbs_exit_code', 'gpu_req',
       'mic_req', 'real_pow', 'ran_alone', 'real_pow_quality', 'n_2_1',
       'n_2_2', 'n_3_1', 'job_tot_timepoints', 'job_timepoints', 'good_nodes',
       'ncores_system', 'njobs_system', 'load_system'],
      dtype='object')

In [271]:
data_set = pd.concat([jobs_train, jobs_test], ignore_index=True)
print(data_set.shape)
ran_alone_data = data_set
#ran_alone_data = data_set[data_set['ran_alone'] == True]
#ran_alone_data = data_set[data_set.node_req*16 == data_set.cpu_req]
ran_alone_data = data_set[(data_set.cpu_req == 8) | (data_set.cpu_req == 16) | (data_set.cpu_req == 24) | (data_set.cpu_req == 32)]
print(ran_alone_data.shape)
data = ran_alone_data
#data = ran_alone_data.sample(10768)
jobs_train_2 = data.head(10000)
jobs_test_2 = data.tail(768)

(81793, 29)
(10237, 29)


In [272]:
jobs_test_2.user.isin(jobs_train.user).shape

(768,)

In [273]:
outfile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset_random"
jobs_train_2.to_csv(outfile_jobs_trainset + ".csv")

outfile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset_random"
jobs_test_2.to_csv(outfile_jobs_testset + ".csv")

In [ ]:
print(jobs[jobs['gpu_req'] != 0].shape[0])
print(jobs[jobs['gpu_req'] > 1].shape[0] / jobs.shape[0])

In [ ]:
jobs[['cpu_req', 'gpu_req']]

In [ ]:
print(jobs['job_timepoints'] / jobs['job_tot_timepoints'])

In [ ]:
jobs[(jobs['job_tot_timepoints'] - jobs['job_timepoints']) <=  0.9 * jobs['job_tot_timepoints']].shape[0]

In [ ]:
(336787 - 26317) / 336787

In [ ]:
26317 / 336787